<a href="https://colab.research.google.com/github/chaen243/SeSAC_colab/blob/main/03_05_FashionMNIST_%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
your_path = "/content/"
try:
    from google.colab import drive
    drive.mount('/content/drive')
    import os
    os.chdir(your_path)
    print(os.getcwd())
except ModuleNotFoundError:
    print("you are not in google colab, pass this phase")
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

Mounted at /content/drive
/content


# Convolutional Neural Network

# FashionMNIST 데이터 학습 시키기

FashionMNIST은 각 이미지에 대해 10개의 클래스를 분류하는 데이터셋이며, 각 이미지는 (28x28) 크기를 가진 RGB 이미지다.

## 데이터 불러오기 및 전처리 파이프라인

파이프라인은 한 데이터 처리 단계의 출력이 다음 단계의 입력으로 이어지는 형태로 연결된 구조를 가리킨다. 이미지 전처리 파이프라인은 `torchvision` 패키지의 `transforms`를 주로 사용한다.

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [3]:

img_transformer = transforms.Compose([
    transforms.ToTensor(),  # Tensor로 변환 (32, 32, 3) > (3, 32, 32)
])

In [4]:
# 데이터 경로
data_path = "./"

train_dataset = datasets.FashionMNIST(
    root=data_path,
    train=True,
    transform=img_transformer,
    download=True)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=64,
    shuffle=True)

test_dataset = datasets.FashionMNIST(
    root=data_path,
    train=False,
    transform=img_transformer,
    download=True)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=64,
    shuffle=True)

100%|██████████| 26.4M/26.4M [00:00<00:00, 115MB/s]


Extracting ./FashionMNIST/raw/train-images-idx3-ubyte.gz to ./FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 5.92MB/s]

Extracting ./FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./FashionMNIST/raw




100%|██████████| 4.42M/4.42M [00:00<00:00, 62.4MB/s]


Extracting ./FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 25.8MB/s]


Extracting ./FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./FashionMNIST/raw



In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [6]:
for batch_idx, (data, target) in enumerate(train_loader):
    # 입력과 타겟 텐서에 GPU 를 사용여부 전달
    data, target = data.to(device), target.to(device)
    break

In [7]:

data.size()

torch.Size([64, 1, 28, 28])

## CNN 모델 (Coding Part)

### 네트워크 설계

* Input Size = (B, 1, 28, 28)
* Output Size = (B, 10)
* Activation Function(`nn.ReLU`): ReLU
* Loss Function(`nn.CrossEntropyLoss`): Cross Entropy Loss
* Optimizer(`optim.Adam`): Adam
* Convolutional Layer: `self.convs` 변수의 `nn.Sequential`안에 구현
    > Conv1
    >
    > 1. Conv(`nn.Conv2d`): 입력채널 1, 출력채널 8, 커널크기 3
    > 2. ReLU(`nn.ReLU`)
    > 3. MaxPool(`nn.MaxPool2d`): 커널크기 2
    >
    > Conv2
    > 4. Conv(`nn.Conv2d`): 입력채널 8, 출력채널 16, 커널크기 3
    > 5. ReLU(`nn.ReLU`)
    > 6. MaxPool(`nn.MaxPool2d`): 커널크기 2
    >
    > Conv3
    > 7. Conv(`nn.Conv2d`): 입력채널 16, 출력채널 32, 커널크기 3
    > 8. ReLU(`nn.ReLU`)
    > 9. MaxPool(`nn.MaxPool2d`): 커널크기 2
    >
    > Conv4
    > 10. Conv(`nn.Conv2d`): 입력채널 32, 출력채널 64, 커널크기 3
    > 11. ReLU(`nn.ReLU`)
* Fully Connected Layer: `self.fc` 변수의 `nn.Sequential`안에 구현
    > 1. Linear(`nn.Linear`): Hidden Size = 100 (**입력 뉴런수 계산 필요**)
    > 2. ReLU(`nn.ReLU`)
    > 3. Linear(`nn.Linear`): Hidden Size = 100
    > 4. ReLU(`nn.ReLU`)
    > 5. Linear(`nn.Linear`): Hidden Size = 10

In [18]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        # 층을 구성
        self.convs = nn.Sequential(
            # Conv1
            nn.Conv2d(in_channels=1,out_channels=8,kernel_size=3,stride=1, padding=1),
            #[B, 1, 28, 28] -> [B, 8, 28, 28]
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # Conv2
            nn.Conv2d(in_channels=8,out_channels=16,kernel_size=3,stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv3
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3,stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv4
            nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=1, padding=1),
            nn.ReLU(),
        )


        self.fc = nn.Sequential( # 분류를 위한 학습층
        nn.Linear(in_features=64*3*3,out_features=100),
        nn.ReLU(),
        nn.Linear(100,100),
        nn.ReLU(),
        nn.Linear(100,10),
        )


    def forward(self, x):
        # forward propagation 수행
        # Conv Layers
        x = self.convs(x)
        # x의 차원을 (미니배치, 32*3*3)에 알맞게 수정한다.
        x = x.view(x.size(0), -1)
        # FC Layers
        x = self.fc(x)
        return x

### 손실함수 및 옵티마이저

In [19]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
# 커스텀 모듈 호출
model = Network().to(device)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

cuda


## 모델훈련

### Train 함수(Coding Part)

In [20]:
def train(model, train_loader, loss_function, optimizer, n_train, print_step, device):
    # Training
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # 입력과 타겟 텐서에 GPU 를 사용여부 전달
        data, target = data.to(device), target.to(device)
                # 경사 초기화
        optimizer.zero_grad()
        # 순방향 전파. 데어터를 넣어 예측값 입력
        output = model(data)
        # 손실값 계산
        loss = loss_function(output, target)
        # 역방향 전파
        loss.backward()
        # 매개변수 업데이트
        optimizer.step()
        # 중간 과정 print
        if batch_idx % print_step == 0:
            percentage = (batch_idx*train_loader.batch_size / n_train) * 100
            print(f" - [{percentage:.2f}%] train loss: {loss:.4f}")

### Validation 함수

In [23]:
def validation(model, test_loader, loss_function, n_test, device):
    # Validation
    model.eval()
    test_loss = 0
    correct = 0
    # torch.no_grad 를 사용하면 requires_grad 를 꺼두게 된다.
    with torch.no_grad():
        for data, target in test_loader:
            # 입력과 타겟 텐서에 GPU 를 사용여부 전달
            data, target = data.to(device), target.to(device)
            # 순방향전파
            output = model(data)
            # 손실값 계산
            test_loss += loss_function(output, target).item()
            # 예측 값에 해당하는 클래스 번호 반환
            pred = output.softmax(1).argmax(dim=1, keepdim=True)
            # 정확하게 예측한 개수를 기록한다
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_accuracy = correct / n_test

    return test_loss, test_accuracy

## 훈련시작

In [24]:
n_train = len(train_dataset)
n_test = len(test_dataset)
n_step = 20
print_step = 300
best_accuracy = 0

for step in range(n_step):
    print(f"[Step] {step+1}/{n_step}\n [Training Step]")
    train(model, train_loader, loss_function, optimizer, n_train, print_step, device)
    test_loss, test_accuracy = validation(model, test_loader, loss_function, n_test, device)
    print(f" [Validation Step]")
    print(f" - test loss: {test_loss:.4f} test accuracy: {test_accuracy*100:.2f} %")
    # 제일 성능을 보인 좋은 모델 저장하기
    if test_accuracy > best_accuracy:
        best_accuracy = test_accuracy
        torch.save(model.state_dict(), "best_model-cifar10.pt")

[Step] 1/20
 [Training Step]
 - [0.00%] train loss: 0.4706
 - [32.00%] train loss: 0.3492
 - [64.00%] train loss: 0.3906
 - [96.00%] train loss: 0.2816
 [Validation Step]
 - test loss: 62.3975 test accuracy: 85.39 %
[Step] 2/20
 [Training Step]
 - [0.00%] train loss: 0.4991
 - [32.00%] train loss: 0.1953
 - [64.00%] train loss: 0.2589
 - [96.00%] train loss: 0.3558
 [Validation Step]
 - test loss: 58.9605 test accuracy: 86.05 %
[Step] 3/20
 [Training Step]
 - [0.00%] train loss: 0.3044
 - [32.00%] train loss: 0.3392
 - [64.00%] train loss: 0.3379
 - [96.00%] train loss: 0.4171
 [Validation Step]
 - test loss: 51.9857 test accuracy: 87.99 %
[Step] 4/20
 [Training Step]
 - [0.00%] train loss: 0.3133
 - [32.00%] train loss: 0.3335
 - [64.00%] train loss: 0.3143
 - [96.00%] train loss: 0.3665
 [Validation Step]
 - test loss: 50.6847 test accuracy: 88.79 %
[Step] 5/20
 [Training Step]
 - [0.00%] train loss: 0.3916
 - [32.00%] train loss: 0.2774
 - [64.00%] train loss: 0.1533
 - [96.00%] tra